# Using LLMs in Python

In [2]:
import os

import dotenv
import PIL.Image

import google.generativeai as genai

In [3]:
loaded: bool = dotenv.load_dotenv()
print("Loaded environment variables." if loaded else "Could not load environment variables.")

Loaded environment variables.


In [7]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [14]:
model_version = 'models/gemini-1.5-flash'
model = genai.GenerativeModel(model_version)
model_info = genai.get_model(model_version)
print(f"Model name: {model_info.name}\nModel version: {model_info.version}\nInput token limit: {model_info.input_token_limit}\nOutput token limit: {model_info.output_token_limit}")

Model name: models/gemini-1.5-flash
Model version: 001
Input token limit: 1000000
Output token limit: 8192


In [15]:
# Counting tokens
prompt = "I want to design a modular apartment building."
model.count_tokens(prompt)

total_tokens: 9

In [16]:
# Prompting
response = model.generate_content(prompt)
print(response.text)

That's a great idea! Designing a modular apartment building offers several advantages, including faster construction, cost-effectiveness, and flexibility. Here's a breakdown to help you get started:

**1. Define Your Target Audience and Goals:**

* **Who are you building for?**  Students, young professionals, families, seniors? Understanding your target market will help you define the building's size, amenities, and layout.
* **What are your design goals?** Sustainability, affordability, modern design, community building?
* **What are your budget constraints?**  Modular construction can be cost-effective, but you'll need to set a realistic budget.

**2.  Modular Building Systems:**

* **Choose a System:**  Research different modular building systems. Some popular options include:
    * **Pre-fabricated modules:**  Entire units are built off-site and shipped to the site for assembly.
    * **Panelized construction:** Large panels containing walls, floors, and ceilings are assembled on-s